# cGAN (conditional GAN) implementation Using Pytorch

## Below is the Todo-list
Will be updated as we go about the code

In [ ]:
#TODO: 

## Implementation Starts here

In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision import transforms
import matplotlib.pyplot as plt
import numpy as np
import torchvision.utils as vutils

In [3]:
# Generator

class Generator(nn.Module):
    def __init__(self, input_size_z = 100, input_size_condition= 100, hidden_size = 256, output_size = 784, device = 'cuda'):
        super().__init__()

        self.device = device

        self.layer = nn.Sequential(
            nn.Linear(input_size_z, 200),
            nn.ReLU(),
        )

        self.layer_2 = nn.Sequential(
            nn.Linear(input_size_condition, 1000),
            nn.ReLU(),
        )

        self.combine = nn.Sequential(
            nn.Linear(1200, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size),
            nn.Sigmoid()
        )

    def forward(self, z, y):
        """
        z: the vector 
        y: the label for the vector
        """
        z = self.layer(z)
        y = self.layer_2(y)

        combined = torch.cat((z, y), 1)

        return self.combine(combined)

In [10]:
def test_generator():
    gen = Generator(input_size_z=100, input_size_condition=10, device='cuda')
    noise = torch.randn(1, 100)
    label = torch.randn(1, 10)

    output = gen(noise, label)
    assert output.shape == (1, 784)
    print("Generator test passed")
test_generator()

Generator test passed


In [ ]:
# Discriminator

class Discriminator(nn.Module):
    def __init__(self, device):
        #TODO: look into maxout layer
        super().__init__()
        
    def forward(self, x):
        pass